# Практическая работа №5

Выполнил студент группы М06-106н Кочетков Дмитрий Андреевич.

## Постановка задачи

Необходимо выполнить все следующие задачи:

1. Сконструировать класс `BinaryHeap`, реализующий следующие операции над двоичной max-кучей:
- `push` - добавление элемента в кучу;
- `pop` – получить значение и удалить корневой элемент из кучи.
- (дополнительное задание – по желанию) Реализовать конструктор класса, которому в качестве параметра передается список элементов, из которых нужно построить кучу.

2. Реализовать класс `TaskQueue` для работы со списком задач. Для каждой задачи задается название и приоритет (число от 1 до 4: 1 – низкий приоритет, 2 – средний приоритет, 3 – высокий приоритет, 4 – критическая задача). Класс должен обеспечивать возможность добавления задач в очередь, извлечение задачи из очереди, смену приоритета задачи из очереди. Для реализации использовать двоичную кучу. Для работы с двоичной кучей использовать библиотеку `heapq`.

3. Сравнить время работы очереди с приоритетом на основе 1) отсортированного списка (list) и 2) двоичной кучи (библиотека `heapq`). Для этого рассчитать время заполнения и извлечения всех данных из очереди с приоритетом с использованием обоих методов (списка и кучи) на списках разной длины. Построить на одном графике зависимости времени работы с очередью от размера списка для обоих реализаций. Сделать вывод.


## Задача 1

### Реализация BinaryHeap

### Тестирование BinaryHeap

## Задача 2

In [2]:
import heapq
from heapq import heappush, heappop

class Task:
    def __init__(self, id, name, priority: int):
        assert isinstance(priority, int)
        self.name = name
        self.priority = priority

class TaskQueue:
    def __init__(self):
        self.tasks = []
        self.next_id = 1

    def insert(self, task: Task):
        heappush(self.tasks, (task.priority, next_id, task))
        next_id += 1

In [3]:
queue = TaskQueue()
queue.insert(Task('Feature', 2))
queue.insert(Task('Bugfix', 1))

## Вывод

В ходе практической работы было реализовано AVL дерево. Выявлена зависимость между количеством узлов дерева и его высотой.